# NDE: Neural Developmental Encodings

In [1]:
# Standard library
from math import isclose
from pathlib import Path

# Third party libraries
import numpy as np

# Local libraries
from ariel import console
from ariel.ec.genotypes.nde import NeuralDevelopmentalEncoding

In [2]:
# --- DATA SETUP ---
CWD = Path.cwd()
DATA = CWD / "__data__"
DATA.mkdir(exist_ok=True)


In [3]:
# --- RANDOM GENERATOR SETUP --- #
SEED = 42
RNG = np.random.default_rng(SEED)

In [4]:
# Global variables
SPAWN_POS = [-0.8, 0, 0.1]
NUM_OF_MODULES = 30
TARGET_POSITION = [5, 0, 0.5]

## Diversity Assessment

### Standard Normal

In [5]:
matrix_history = []
for _i in range(500):
    genotype_size = 64
    type_p_genes = RNG.random(genotype_size).astype(np.float32)
    conn_p_genes = RNG.random(genotype_size).astype(np.float32)
    rot_p_genes = RNG.random(genotype_size).astype(np.float32)

    genotype = [
        type_p_genes,
        conn_p_genes,
        rot_p_genes,
    ]

    nde = NeuralDevelopmentalEncoding(number_of_modules=NUM_OF_MODULES)
    p_matrices = nde.forward(genotype)
    matrix_history.append(np.concatenate([p.flatten() for p in p_matrices]))

sum_of_diff = 0
for i in range(len(matrix_history)):
    for j in range(i, len(matrix_history)):
        if i != j:
            sum_of_diff += np.sum(np.abs(matrix_history[i] - matrix_history[j]))

mean = sum_of_diff / (len(matrix_history) * (len(matrix_history) - 1))
console.print(f"[bold green]Mean diversity:[/bold green] {mean}")

Mean diversity: 51.12075424194336

In [6]:
matrix_history = []
scale = 8192
samples = 500
for _ in range(samples):
    genotype_size = 64
    type_p_genes = RNG.uniform(-scale, scale, genotype_size).astype(
        np.float32,
    )
    conn_p_genes = RNG.uniform(-scale, scale, genotype_size).astype(
        np.float32,
    )
    rot_p_genes = RNG.uniform(-scale, scale, genotype_size).astype(
        np.float32,
    )

    genotype = [
        type_p_genes,
        conn_p_genes,
        rot_p_genes,
    ]

    nde = NeuralDevelopmentalEncoding(number_of_modules=NUM_OF_MODULES)
    p_matrices = nde.forward(genotype)
    matrix_history.append(np.concatenate([p.flatten() for p in p_matrices]))

sum_of_diff = 0
for i in range(len(matrix_history)):
    for j in range(i, len(matrix_history)):
        if i != j:
            sum_of_diff += np.sum(np.abs(matrix_history[i] - matrix_history[j]))

mean = sum_of_diff / (len(matrix_history) * (len(matrix_history) - 1))
console.print(f"[bold green]Mean diversity:[/bold green] {mean}")

Mean diversity: 1439.3802490234375

### Find Maximal Diversity

In [7]:
old_diversity = 0
new_diversity = -1
scale = 1
samples = 500
while not isclose(old_diversity, new_diversity):
    scale *= 2
    matrix_history = []
    old_diversity = new_diversity

    # Generate samples
    console.log(f"Sampling with scale: {scale}")
    for _ in range(samples):
        genotype_size = 64 * 4
        type_p_genes = RNG.uniform(-scale, scale, genotype_size).astype(
            np.float32,
        )
        conn_p_genes = RNG.uniform(-scale, scale, genotype_size).astype(
            np.float32,
        )
        rot_p_genes = RNG.uniform(-scale, scale, genotype_size).astype(
            np.float32,
        )

        genotype = [
            type_p_genes,
            conn_p_genes,
            rot_p_genes,
        ]

        nde = NeuralDevelopmentalEncoding(
            number_of_modules=NUM_OF_MODULES,
            genotype_size=genotype_size,
        )
        p_matrices = nde.forward(genotype)
        matrix_history.append(np.concatenate([p.flatten() for p in p_matrices]))

    # Calculate diversity
    sum_of_diff = 0
    for i in range(len(matrix_history)):
        for j in range(i, len(matrix_history)):
            if i != j:
                sum_of_diff += np.sum(
                    np.abs(matrix_history[i] - matrix_history[j]),
                )

    # New diversity
    mean = sum_of_diff / (len(matrix_history) * (len(matrix_history) - 1))
    console.print(f"[bold green]Mean diversity:[/bold green] {mean}")
    new_diversity = int(mean)

[16:52:22] Sampling with scale: 2                                                                   ]8;id=265350;file:///var/folders/x7/1fxq57_x2d1c5hmfwj4l43cw0000gp/T/ipykernel_25063/416473013.py\416473013.py]8;;\:]8;id=196787;file:///var/folders/x7/1fxq57_x2d1c5hmfwj4l43cw0000gp/T/ipykernel_25063/416473013.py#11\11]8;;\

Mean diversity: 52.79603576660156

[16:52:23] Sampling with scale: 4                                                                   ]8;id=44574;file:///var/folders/x7/1fxq57_x2d1c5hmfwj4l43cw0000gp/T/ipykernel_25063/416473013.py\416473013.py]8;;\:]8;id=541094;file:///var/folders/x7/1fxq57_x2d1c5hmfwj4l43cw0000gp/T/ipykernel_25063/416473013.py#11\11]8;;\

Mean diversity: 58.3109016418457

[16:52:25] Sampling with scale: 8                                                                   ]8;id=793328;file:///var/folders/x7/1fxq57_x2d1c5hmfwj4l43cw0000gp/T/ipykernel_25063/416473013.py\416473013.py]8;;\:]8;id=818785;file:///var/folders/x7/1fxq57_x2d1c5hmfwj4l43cw0000gp/T/ipykernel_25063/416473013.py#11\11]8;;\

...

Mean diversity: 1437.289306640625

[16:52:35] Sampling with scale: 8192                                                                ]8;id=447912;file:///var/folders/x7/1fxq57_x2d1c5hmfwj4l43cw0000gp/T/ipykernel_25063/416473013.py\416473013.py]8;;\:]8;id=729370;file:///var/folders/x7/1fxq57_x2d1c5hmfwj4l43cw0000gp/T/ipykernel_25063/416473013.py#11\11]8;;\

Mean diversity: 1439.3741455078125

[16:52:36] Sampling with scale: 16384                                                               ]8;id=346476;file:///var/folders/x7/1fxq57_x2d1c5hmfwj4l43cw0000gp/T/ipykernel_25063/416473013.py\416473013.py]8;;\:]8;id=31897;file:///var/folders/x7/1fxq57_x2d1c5hmfwj4l43cw0000gp/T/ipykernel_25063/416473013.py#11\11]8;;\

Mean diversity: 1439.8330078125